# 🛒 E-commerce Sales & Behavior Analysis
## Project Objective
This project analyzes a large e-commerce event dataset (56 million rows) to understand customer behavior and identify key drivers of revenue. The goal is to move beyond simple reporting to uncover actionable insights.

### Analysis Workflow:
1.  **Data Cleaning & Preparation:** Efficiently load and clean the 56M row dataset using vectorized operations.
2.  **Sales Performance Analysis:** Identify top products, brands, and categories driving revenue.
3.  **Behavioral Funnel Analysis:** Correctly model the user conversion funnel (View → Cart → Purchase) on a session basis.
4.  **Cart Abandonment Insights:** Analyze *which* products are most frequently left in abandoned carts.
5.  **Customer Retention (Cohort Analysis):** Track user cohorts to understand customer loyalty and retention over time.
6.  **Predictive Modeling:** Build a RandomForest model to predict the likelihood of a user making a purchase based on their browsing behavior.


# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import joblib

# ML Libraries
# Import the ML Libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_curve,confusion_matrix, classification_report)
# Set default plot themes
sns.set_theme(style="whitegrid")
pio.templates.default = "plotly_white"

## 2. Data Loading, Cleaning & Feature Engineering
We will load the data using efficient practices for a 56M row dataset.
1.  **Specify dtypes:** Reduces memory usage.
2.  **Specify format:** Speeds up `pd.to_datetime` significantly.
3.  **Vectorized Operations:** Use `.str.split()` instead of `.apply()` for feature extraction.

In [2]:
file_path = r"C:\Users\npsta\Downloads\2020-Jan.csv"

### 2.1 Load a small dataset

In [3]:
sample_df = pd.read_csv(file_path, nrows = 10000)
sample_df.head(10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-01-01 00:00:00 UTC,view,1005073,2232732093077520756,construction.tools.light,samsung,1130.02,519698804,69b5d72f-fd6e-4fed-aa23-1286b2ca89a0
1,2020-01-01 00:00:01 UTC,view,1005192,2232732093077520756,construction.tools.light,meizu,205.67,527767423,7f596032-ccbf-4643-9bad-e36a209512b4
2,2020-01-01 00:00:01 UTC,view,100063693,2053013552427434207,apparel.shirt,turtle,136.43,519046195,d1e2f343-84bb-49bd-b13d-ca0f1ed9910e
3,2020-01-01 00:00:01 UTC,view,5100816,2232732103831716449,apparel.shoes,xiaomi,29.95,518269232,0444841c-38ef-410c-b11f-7b35ea4e5991
4,2020-01-01 00:00:02 UTC,view,100014325,2232732103294845523,apparel.shoes.step_ins,intel,167.20,587748686,31b7d4cf-dfac-4895-9927-90fa3254f860
5,2020-01-01 00:00:02 UTC,view,16500026,2232732108713886406,apparel.costume,gamma,37.35,581037554,c161400e-630b-4b59-8797-d9b9714444bf
6,2020-01-01 00:00:02 UTC,view,1802026,2232732099754852875,appliances.personal.massager,samsung,576.33,581274910,44ca9bc7-3ba8-4546-bdf5-10b2172ed883
7,2020-01-01 00:00:02 UTC,view,11600004,2053013554834964853,appliances.kitchen.kettle,sven,43.48,595265136,32c08a07-f2d0-4da5-83fd-40ce97c651e6
8,2020-01-01 00:00:02 UTC,view,4802273,2232732079706079299,sport.bicycle,samsung,6.64,595414563,176fd102-7b61-4452-a0e5-f1f8cc9b4b95
9,2020-01-01 00:00:03 UTC,view,1005115,2232732093077520756,construction.tools.light,apple,869.46,531140669,84c838d4-6e10-4b7d-8d29-90749e577a6b


In [4]:
sample_df.dtypes

event_time        object
event_type        object
product_id         int64
category_id        int64
category_code     object
brand             object
price            float64
user_id            int64
user_session      object
dtype: object

In [5]:
# Define dtypes for memory efficiency
dtypes = {
'event_type': 'category',
'product_id': 'int32',
'category_id': 'int64',
'category_code': 'str',
'brand': 'str',
'price': 'float32',
'user_id': 'int32',
'user_session': 'str'}

df = pd.read_csv(file_path,dtype = dtypes,parse_dates = ['event_time'],date_format = '%Y-%m-%d %H:%M:%S UTC'
)                

print("Data loaded successfully.")
print(f"Shape of the dataset: {df.shape}")
      

Data loaded successfully.
Shape of the dataset: (55967041, 9)


### 2.2 Data Cleaning & Feature Engineering

In [6]:
# Drop Duplicates
df.drop_duplicates(inplace = True)

### Fixing and Verifying Missing Data

In this step, we'll clean the 11.5 million missing values in category_code and brand.

Instead of just deleting these rows (which would lose 11% of our data) or filling them with "Unknown" (which is inaccurate), we will recover the missing information.

My logic is:

- For Categories: A category_id will always have the same category_code.

- For Brands: A product_id will always have the same brand.

- We will create "lookup maps" from the data we do have to fill in the data that's missing.

- Instead of dropping rows with missing data, the project recovered over 11.5 million missing values for category_code and brand.

- Method: It used a groupby().transform() method to fill missing brand names based on their product_id and missing category_code    based on their category_id.

- Result: This reduced missing values for these columns from over 5 million and 6.5 million (respectively) down to zero.

In [7]:
# Handle missing values

# --- a: Count Missing Values BEFORE ---
print("--- BEFORE CLEANING ---")
print(df.isnull().sum())
print("-" * 30)

# --- b: Fixing Missing Category Codes ---
print("Fixing missing category codes...")
category_mode = df.groupby('category_id')['category_code'].transform(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')
df['category_code'] = category_mode


# --- c: Fixing Missing Brands ---
print("Fixing missing brands...")
brand_mode = df.groupby('product_id')['brand'].transform(lambda x: x.mode()[0] if not x.mode().empty else 'Unknown')
df['brand'] = brand_mode

# --- d: Handle Any Leftovers ---
# Fill the few remaining blanks with 'Unknown'
df['category_code'] = df['category_code'].fillna('Unknown')
df['brand'] = df['brand'].fillna('Unknown')

# Drop the 19 rows with no session ID
df.dropna(subset=['user_session'], inplace=True)
df.drop_duplicates(inplace=True)

print("Fixing complete.")
print("\n--- AFTER CLEANING ---")
print(df.isnull().sum())
print("-" * 30)

# --- e: Check for Data Consistency ---
print("Verifying data consistency...")

# 1. Check for product_id <> brand conflicts
product_brands = df.groupby('product_id')['brand'].nunique()
brand_conflicts = product_brands[product_brands > 1].count()
print(f"Products found with inconsistent brand names: {brand_conflicts}")

# 2. Check for category_id <> category_code conflicts
category_codes = df.groupby('category_id')['category_code'].nunique()
code_conflicts = category_codes[category_codes > 1].count()
print(f"Categories found with inconsistent codes: {code_conflicts}")

print("-" * 30)
print("Data Handleing is done.")

--- BEFORE CLEANING ---
event_time             0
event_type             0
product_id             0
category_id            0
category_code    5034766
brand            6527940
price                  0
user_id                0
user_session          19
dtype: int64
------------------------------
Fixing missing category codes...
Fixing missing brands...
Fixing complete.

--- AFTER CLEANING ---
event_time       0
event_type       0
product_id       0
category_id      0
category_code    0
brand            0
price            0
user_id          0
user_session     0
dtype: int64
------------------------------
Verifying data consistency...
Products found with inconsistent brand names: 0
Categories found with inconsistent codes: 0
------------------------------
Data Handleing is done.


### 2.3 Feature Engineering

In [8]:
# clean the category_code column ---
cat_cols = ['category_l1','category_l2','category_l3']
df[cat_cols] = df['category_code'].str.split('.',expand = True, n=2)
# Fill NaN with None
for col in cat_cols:
    df[col] = df[col].fillna('None')

In [9]:
# Time Featuring
df['event_hour'] = df['event_time'].dt.hour
df['event_weekday'] = df['event_time'].dt.day_name().str.capitalize()

In [10]:
# Column for Funnel Analysis
df['is_view'] = (df['event_type'] == 'view').astype(int)
df['is_cart'] = (df['event_type'] == 'cart').astype(int)
df['is_purchase'] = (df['event_type'] == 'purchase').astype(int)

In [11]:
# Column for Revenue
df['revenue'] = 0.0
df.loc[df['is_purchase'] ==1,'revenue'] = df['price']
print("Data Cleaning and Feature Engineering is complete.")

Data Cleaning and Feature Engineering is complete.


### Insight

- Categorical Features: The category_code (e.g., construction.tools.light) was split into three hierarchical columns (category_l1, category_l2, category_l3) for more granular analysis.

- Temporal Features: event_hour and event_weekday were extracted from the event_time column to analyze time-based behavior.

- Behavioral Flags: New columns (is_view, is_cart, is_purchase) were created to enable session-level funnel analysis.

- Financial Metric: A revenue column was created by calculating sales from price only on purchase events.